# Лабораторная работа №9

**Тема: "Дифференцирование и интегрирование"**

Работу выполнил

студент группы 5130901/20201

Нестеренко Сергей Андреевич

---

## Упражнение 9.1

## Упражнение 9.2

## Упражнение 9.3

## Упражнение 9.4

## Упражнение 9.5